### 0. Imports

In [319]:
import pandas as pd
import numpy as np

### 1. Introduction to this notebook

# 2. Transformation

## 2.1 Flights

In [322]:
itineraries = pd.read_parquet("../data/flights/itineraries.parquet")
itineraries.info()
itineraries.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_query                  349 non-null    datetime64[ns]
 1   score                       349 non-null    float64       
 2   duration                    349 non-null    int64         
 3   price                       349 non-null    int64         
 4   price_currency              349 non-null    object        
 5   stops                       349 non-null    int64         
 6   departure                   349 non-null    datetime64[ns]
 7   arrival                     349 non-null    datetime64[ns]
 8   company                     349 non-null    object        
 9   self_transfer               349 non-null    bool          
 10  fare_isChangeAllowed        349 non-null    bool          
 11  fare_isPartiallyChangeable  349 non-null    bool          

,date_query,score,duration,price,price_currency,stops,departure,arrival,company,self_transfer,fare_isChangeAllowed,fare_isPartiallyChangeable,fare_isCancellationAllowed,fare_isPartiallyRefundable,origin_airport,destination_airport
0,2024-11-02 19:10:25.310576,0.999000,85,100,€,0,2024-11-03 10:30:00,2024-11-03 11:55:00,Iberia,False,False,False,False,False,Barcelona,Madrid
1,2024-11-02 19:10:25.313576,0.694966,85,110,€,0,2024-11-03 11:50:00,2024-11-03 13:15:00,Vueling Airlines,False,False,False,False,False,Barcelona,Madrid
2,2024-11-02 19:10:25.314575,0.564122,90,139,€,0,2024-11-03 20:30:00,2024-11-03 22:00:00,Air Europa,False,False,False,False,False,Barcelona,Madrid
3,2024-11-02 19:10:25.315574,0.432498,90,142,€,0,2024-11-03 11:50:00,2024-11-03 13:20:00,Air Europa,False,False,False,False,False,Barcelona,Madrid
4,2024-11-02 19:10:25.316572,0.425545,85,168,€,0,2024-11-03 13:30:00,2024-11-03 14:55:00,Iberia,False,False,False,False,False,Barcelona,Madrid


Apparently, all data types and values come out correctly from the Air Scraper API. Something that is missing and would have to be provided to the user would be the suggested url to get the flights.

## 2.2 Accommodations

In [343]:
booking_df = pd.read_parquet("../data/accommodations/booking.parquet")
booking_df.info()
booking_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   query_date               754 non-null    datetime64[ns]
 1   checkin                  754 non-null    object        
 2   checkout                 754 non-null    object        
 3   n_adults_search          754 non-null    object        
 4   n_children_search        754 non-null    object        
 5   n_rooms_search           754 non-null    object        
 6   name                     754 non-null    object        
 7   url                      754 non-null    object        
 8   price_currency           754 non-null    object        
 9   total_price_amount       754 non-null    object        
 10  distance_city_center_km  754 non-null    object        
 11  score                    732 non-null    object        
 12  n_comments               732 non-nul

,query_date,checkin,checkout,n_adults_search,n_children_search,n_rooms_search,name,url,price_currency,total_price_amount,...,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi
0,2024-11-02 20:38:04.015543,2024-11-02,2024-11-04,2,0,1,Catalonia Albeniz,https://www.booking.com/hotel/es/cataloniaalbe...,€,262,...,Yes,Yes,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,None,No
1,2024-11-02 20:38:04.044726,2024-11-02,2024-11-04,2,0,1,Catalonia Sagrada Familia,https://www.booking.com/hotel/es/cataloniaarag...,€,292,...,Yes,Yes,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,None,No
2,2024-11-02 20:38:04.145106,2024-11-02,2024-11-04,2,0,1,Catalonia Atenas,https://www.booking.com/hotel/es/cataloniatena...,€,285,...,Yes,Yes,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,None,No
3,2024-11-02 20:38:04.150123,2024-11-02,2024-11-04,2,0,1,Catalonia Park Güell,https://www.booking.com/hotel/es/catalonia-par...,€,222,...,Yes,Yes,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,None,No
4,2024-11-02 20:38:04.154201,2024-11-02,2024-11-04,2,0,1,Roma Reial,https://www.booking.com/hotel/es/roma-reial.es...,€,232,...,Yes,No,Habitación Doble - 2 camas,No,Yes,No,No,No,9.4,No


Convert checkin and checkout to datetime

In [344]:
booking_df[["checkin","checkout"]] = booking_df[["checkin","checkout"]].astype("datetime64[ns]")

In [345]:
booking_df["room_type"].value_counts()[:20]

room_type
Habitación Doble - 1 o 2 camas                        75
Habitación Doble                                      66
Habitación Doble - 2 camas                            46
Habitación Doble con baño compartido                  40
Cama en habitación compartida mixta de 10 camas       29
Habitación Doble con baño compartido - 2 camas        19
Cama en habitación compartida mixta de 6 camas        17
Habitación Doble Económica                            16
Habitación Doble Superior                             12
Habitación Doble Estándar - 1 o 2 camas               12
Cama en habitación compartida mixta de 8 camas        12
Habitación Doble con baño privado - 2 camas           11
Habitación Doble con baño privado                     11
Apartamento de 1 dormitorio                           11
Habitación (1 o 2 adultos) - 1 o 2 camas               8
Habitación Doble con balcón - 2 camas                  7
Habitación Doble Pequeña                               7
Cama en habitación co

Standardize room types

In [346]:
conditions = [
    booking_df["room_type"].str.startswith("Apartamento"),
    booking_df["room_type"].str.startswith("Cama en habitacion compartida"),
    booking_df["room_type"].str.startswith("Habitacion Doble Superior"),
    booking_df["room_type"].str.startswith("Habitacion Doble")
]

choices = [
    "Apartamento",
    booking_df["room_type"],  # keeps the original value for "Cama en habitacion compartida"
    "Habitacion Doble Superior",
    "Habitacion Doble"
]


booking_df["standardized_room_type"] = np.select(conditions, choices, default=booking_df["room_type"])

Apartamento
Cama en habitacion compartida
Habitacion Doble Superior
Habitacion Doble 

In [347]:
booking_df["shared_bathroom"] = np.where(booking_df["room_type"].str.contains("baño compartido"),"Yes","No")

Add separate info about balcony

In [348]:
booking_df["balcony"] = np.where(booking_df["room_type"].str.contains("balcón"),"Yes","No")

In [349]:
booking_df["sustainability_cert"].value_counts()

sustainability_cert
No     648
Yes    106
Name: count, dtype: int64

In [350]:
booking_df["free_cancellation"].value_counts()

free_cancellation
No     601
Yes    153
Name: count, dtype: int64

In [352]:
booking_df["breakfast_included"].value_counts()

breakfast_included
No     601
Yes    153
Name: count, dtype: int64

## 2.3 Activities

In [ ]:
activities_df = pd.read_parquet("../data/activities/90_day_availability.parquet")

In [323]:
activities_df

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish,address,paired,last_day_of_month_start
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español,"Pg. de Gràcia, 92, L'Eixample, 08008 Barcelona...","[(29, []), (02, []), (01, []), (28, []), (03, ...",30
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español,"Estació del Nord, L'Eixample, 08013 Barcelona,...","[(27, [10:00, 11:00, 12:15, 13:15, 16:00, 16:3...",30
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español,"Moll de les Drassanes, 3P, Ciutat Vella, 08039...","[(28, [16:30]), (27, [16:30]), (03, [16:30]), ...",30
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None,"Avinguda Miramar, 30, Sants-Montjuïc, 08038 Ba...","[(27, [10:00]), (01, [10:00]), (30, [10:00]), ...",30
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español,"Ctra. del Carmel, 23, Horta-Guinardó, 08024 Ba...","[(02, [11:30]), (29, [11:30]), (28, [11:30]), ...",30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2024-11-02 14:13:58.778257,2024-12-03,2024-12-09,Tour de los misterios y leyendas de Barcelona,En esta ruta nocturna por Barcelona descubrire...,www.civitatis.com/es/barcelona/tour-misterios-...,www.civitatis.com/f/espana/barcelona/tour-mist...,None,None,1h 30m,41.387471,2.172001,4.20,EUR,Visitas guiadas y free tours,Español,"Carrer d'Estruc, 4, Ciutat Vella, 08010 Barcel...","[(-, -)]",31
260,2024-11-02 14:13:58.779258,2024-12-03,2024-12-09,Entrada al Spotify Camp Nou,Si queréis visitar el estadio del Fútbol Club ...,www.civitatis.com/es/barcelona/camp-nou-experi...,www.civitatis.com/f/espana/barcelona/camp-nou-...,None,None,1h 30m,41.380053,2.121457,2.91,EUR,Entradas,Español,"94JC+2H Barcelona, Spain","[(-, -)]",31
261,2024-11-02 14:13:58.780262,2024-12-03,2024-12-09,Free tour de los misterios y leyendas del Barr...,Acompañadnos en este free tour por Barcelona e...,www.civitatis.com/es/barcelona/free-tour-miste...,www.civitatis.com/f/espana/barcelona/free-tour...,None,None,2 horas,41.3838057,2.1757634,2.00,EUR,Visitas guiadas y free tours,Español,"Carrer de Santa Llúcia, 1, Ciutat Vella, 08002...","[(-, -)]",31
262,2024-11-02 14:13:58.781265,2024-12-03,2024-12-09,"Excursión a Gerona, Figueras y Museo Dalí",En esta excursión a Gerona y Figueras conocere...,www.c

### 2.3.1 Cleaning neccessary

1. Remove duplicates
2. Keep only the good image: The second if it has 2, otherwise the first.
3. Take unique, non null and non zero latitudes and longitudes, to convert to address.
4. Format the available dates and times

Careful as some non-available times might mean they do not apply, i.e. tickets.

##### 2.3.1.1 Remove duplicates

By activity_name and activity_date_range_start

In [123]:
activities_df[activities_df.duplicated(subset=["activity_date_range_start","activity_name"],keep=False)].sort_values(by="activity_name").head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
226,2024-11-02 14:13:58.688766,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 09, 03, 05, 08, 07]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
235,2024-11-02 14:13:58.724155,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 07, 05, 03, 08, 09]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
234,2024-11-02 14:13:58.706054,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 08, 09, 06, 04, 05, 03]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
243,2024-11-02 14:13:58.738170,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 04, 06, 09, 03, 05, 08]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
237,2024-11-02 14:13:58.730169,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao de Carmen,¿Queréis experimentar lo mejor de la cultura e...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[05, 09, 04, 07, 03]","[[], [], [], [], []]",1 hora,41.3689218,2.1488681,17.00,EUR,Espectáculos,None


In [124]:
activities_df.drop_duplicates(subset=["activity_date_range_start","activity_name"], inplace=True)

#### 2.3.1.2 Keep only the good image: The second if it has 2, otherwise the first.

In [125]:
activities_df["image"] = np.where(~activities_df["image2"].isna(), activities_df["image2"], activities_df["image"])
activities_df.drop(columns="image2",inplace=True)

In [ ]:
activities_df.head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español


#### 2.3.1.3 Take unique, non null and non zero latitudes and longitudes, to convert to address.

In [127]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [128]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [129]:
import aiohttp
import asyncio

async def get_address(lat: float, lon: float, api_key: str) -> str:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
            
            if data.get("status") == "OK":
                address = data["results"][0]["formatted_address"]
                return address
            else:
                return "Address not found"

# Example usage
async def get_adresses_async(lat_lon_df):
    api_key = GOOGLE_API_KEY
    lat_lon_tasks = [get_address(lat, lon, api_key) for lat, lon in lat_lon_df.itertuples(index=False, name=None)]
    addresses = await asyncio.gather(*lat_lon_tasks)
    return addresses


In [130]:
not_nan = activities_df[["latitude", "longitude"]].notna().all(axis=1)
unique_combinations = activities_df.loc[not_nan,["latitude","longitude"]].drop_duplicates()

In [131]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [132]:
unique_combinations["lat_lon"] = list(zip(unique_combinations.latitude, unique_combinations.longitude))
activities_df["lat_lon"] = list(zip(activities_df.latitude, activities_df.longitude))

In [133]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [134]:
lat_lon_to_address_ampping = dict(zip(unique_combinations["lat_lon"], unique_combinations["address"]))

In [135]:
activities_df["address"] = activities_df["lat_lon"].map(lat_lon_to_address_ampping)
activities_df.drop(columns="lat_lon",inplace=True)

In [136]:
activities_df.head(5)

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish,address
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español,"Pg. de Gràcia, 92, L'Eixample, 08008 Barcelona..."
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español,"Estació del Nord, L'Eixample, 08013 Barcelona,..."
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español,"Moll de les Drassanes, 3P, Ciutat Vella, 08039..."
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None,"Avinguda Miramar, 30, Sants-Montjuïc, 08038 Ba..."
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español,"Ctra. del Carmel, 23, Horta-Guinardó, 08024 Ba..."


#### 2.3.1.4 Format the available dates and times

Available dates and times for each activity will have to go to another table to ensure database normalisation. To do that, it will be necessary to explode the df twice. 

Then, to complete the date of availability, some rules for transformation have to be established:

In [307]:
availability_activities = activities_df[["available_days","available_times","activity_name","activity_date_range_start","activity_date_range_end"]]
availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_12864\3428466598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)


In [308]:
availability_activities = availability_activities.explode("paired", ignore_index=True) #.drop(columns=["available_days","available_times"])

In [309]:
availability_activities.head()

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end,paired
0,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(29, [])"
1,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(02, [])"
2,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(01, [])"
3,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(28, [])"
4,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(03, [])"


Now, there is a table with the 

In [310]:
availability_activities[["available_days", "available_times"]] = pd.DataFrame(availability_activities["paired"].tolist(), index=availability_activities.index)
availability_activities = availability_activities.drop(columns="paired")
availability_activities

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end
0,29,[],Entrada a La Pedrera,2024-11-27,2024-12-03
1,02,[],Entrada a La Pedrera,2024-11-27,2024-12-03
2,01,[],Entrada a La Pedrera,2024-11-27,2024-12-03
3,28,[],Entrada a La Pedrera,2024-11-27,2024-12-03
4,03,[],Entrada a La Pedrera,2024-11-27,2024-12-03
...,...,...,...,...,...
1272,-,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09
1273,-,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09
1274,-,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09
1275,-,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09


In [311]:
availability_activities = availability_activities.explode("available_times")
availability_activities

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end
0,29,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
1,02,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
2,01,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
3,28,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
4,03,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
...,...,...,...,...,...
1272,-,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09
1273,-,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09
1274,-,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09
1275,-,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09


In [ ]:
import datetime

In [ ]:
availability_activities["activity_date_range_start"] = pd.to_datetime(availability_activities["activity_date_range_start"])
availability_activities["activity_date_range_end"] = pd.to_datetime(availability_activities["activity_date_range_end"])

availability_activities[["available_days","available_times"]] = availability_activities[["available_days","available_times"]].replace("-",np.nan)

availability_activities["available_times"] = availability_activities["available_times"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M").time() if pd.notna(x) else pd.NaT)


def calculate_adjusted_date(row):
    # return Not a Time if nan
    if pd.isna(row["available_days"]):
        return pd.NaT
    
    # convert from string
    day = int(row["available_days"]) 

    # if available date is bigger than end range, belongs to first month. Else second.
    if day > row["activity_date_range_end"].day:
        return row["activity_date_range_start"].replace(day=day)
    else:
        return row["activity_date_range_end"].replace(day=day)


availability_activities["available_date"] = availability_activities.apply(calculate_adjusted_date, axis=1)

availability_activities.drop(columns="available_days",inplace=True)



In [ ]:
availability_activities.dtypes

available_times                      object
activity_name                        object
activity_date_range_start    datetime64[ns]
activity_date_range_end      datetime64[ns]
available_date               datetime64[ns]
dtype: object